# Use Trapezoid, Simpson's Rule, and Romberg integration

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Define the function to integrate

In [ ]:
def func(x):
    a = -2.0
    b = 10.0
    
    return (np.exp(a*x))*np.cos(b*x)

# Define the integral to check answer

In [ ]:
def func_integral(x):
    return np.exp(-2*x)*(5*np.sin(10*x)-np.cos(10*x))/52

# Define the core of the trapezoid method

In [ ]:
def trapezoid_core(f, x, h):
    return 0.5*h*(f(x+h) + f(x))

# Define wrapper function to perform trapezoid method

In [ ]:
def trapezoid_method(f,a,b,N):
    
    # f is function to integrate
    # a is lower limit
    # b is upper limit
    # N is the number of chunks to use
    
    # define x values for integral
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # define the value of the integral
    Fint = 0.0
    
    # perform the integral using the trapezoid method
    for i in range(0,N-1,1):
        Fint += trapezoid_core(f,x[i],h)
    
    # return the answer
    return Fint

# Define the core of Simpson's method

In [ ]:
def simpsons_core(f, x, h):
    return h*( f(x) + 4.*f(x+h) + f(x+2*h))/3.

# Define a wrapper function for Simpson's method

In [ ]:
def simpsons_method(f,a,b,N):
    
    # f is function to integrate
    # a is lower limit
    # b is upper limit
    # N is the number of function evaluations to use
    # note: number of chunks will be N-1
    
    # define x values for integral
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # define the value of the integral
    Fint = 0.0
    
    # perform the integral using Simpson's method
    for i in range(0,N-2,2):
        Fint += simpsons_core(f,x[i],h)
        
    # apply simpson's over the last interval if N is even
    if ((N%2)==0):
        Fint += simpsons_core(f, x[-2],0.5*h)
    
    # return the answer
    return Fint

# Define the romberg integration core

In [ ]:
def romberg_core(f,a,b,i):
    
    # find the width of the interval
    h = b-a
    
    # set the increment between new function evaluations
    dh = h/2.**(i)
    
    # set the cofactor
    K = h/2.**(i+1)
    
    # sum the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
    
    return K*M

# Define a wrapper function for Romberg

In [ ]:
def romberg_integration(f,a,b,tol):
    
    i = 0
    
    # define max number of iterations
    imax = 1000
    
    # Define an error estimate, set to large value
    delta = 100.0*np.fabs(tol)
    
    # set an array of integral answers
    I = np.zeros(imax,dtype=float)
    
    # get zeroth romberg iteration
    I[0] = 0.5*(b-a)* (f(a) + f(b))
    
    # iterate by 1
    i += 1
    
    while(delta>tol):
        
        # find romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        # compute new fractional error estimate
        delta = np.fabs((I[i] - I[i-1])/I[i])
        
        print (i, I[i],I[i-1],delta)
        if (i >= 20):
            print("This could take a few mintues, please wait..")
        
        if (delta>tol):
            i+=1
            if(i>imax):
                print("max iterations reached")
                raise StopIteration('Stopping iterations after ',i)
    
    return I[i], i

# Perform the integration using each method

In [ ]:
lower_bnd = 0.0
upper_bnd = np.pi
tolerance = 1.0e-6

Answer = func_integral(upper_bnd) - func_integral(lower_bnd)
print("True Answer: ", Answer)



# perfrom Trapezoid integration

# set variable for trapezoid answer to a high value
trap_ans = 10000.

trap_intervals = 1

while (np.fabs(Answer - trap_ans) > tolerance):
    trap_intervals += 1
    trap_ans = trapezoid_method(func, lower_bnd, upper_bnd, trap_intervals)

print("Answer from Trapezoid method: ", trap_ans, " Required intervals: ", trap_intervals)




# perform Simpson's method integration

# set variable for Simpson's method answer to a high value
simpsons_ans = 10000.

simpsons_intervals = 1

while (np.fabs(Answer - simpsons_ans) > tolerance):
    simpsons_intervals += 1
    simpsons_ans = simpsons_method(func, lower_bnd, upper_bnd, simpsons_intervals)

print("Answer from Simpson's method: ", simpsons_ans, " Required intervals: ", simpsons_intervals)



# perform the Romberg method integration

romberg_ans, romberg_intervals = romberg_integration(func, lower_bnd, upper_bnd, tolerance)

print("Answer from Romberg method: ", romberg_ans, " Iterations required: ", romberg_intervals)